In [9]:
import pandas as pd
import numpy as np
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.decomposition import TruncatedSVD

#### Opening and Cleaning Data

In [10]:
df = pd.read_csv("dummy_data.csv")
# TODO: create a data cleaning step, allowing more flexible data cleaning techniques
corpus = df["Description"].dropna().drop_duplicates().to_numpy()

#### Preprocessing: Vectorizing Corpus and Query

In [11]:
def vectorize(corpus, refit=False):
    """
    Expects a list of strings to be vectorized into TF-IDF vectors, which are transformed into SVD vectors.
    
    Parameters:
    corpus (list): a list of strings containing the corpus to be vectorized.
    refit (bool): if set to True it will refit TfidfVectorizer and TruncatedSVD using the entire corpus. 
  
    Returns:
    np.array: the vectorized and transformed vectors
    """
    # TODO: use kwargs to allow parameters customization of TfidfVectorizer and TruncatedSVD
    if refit:
        vectorizer = TfidfVectorizer(
            ngram_range=(1,3),
            token_pattern="\w+", # get only words
            sublinear_tf=True, # to normalize vectors   
        )
        
        vectors = vectorizer.fit_transform(corpus).todense()
        
        svd = TruncatedSVD(n_components = 200) # to speed up similarity calculation
        transformed = svd.fit_transform(vectors)
        
        with open("vectorizer.pkl", "wb") as f:
            pickle.dump(vectorizer, f)
        with open("svd.pkl", "wb") as f:
            pickle.dump(svd, f)
            
    else:
        with open("vectorizer.pkl", "rb") as f:
            vectorizer = pickle.load(f)
        with open("svd.pkl", "rb") as f:
            svd = pickle.load(f)
            
        vectors = vectorizer.transform(corpus).todense()
        transformed = svd.transform(vectors)

    # TODO: plot singular values to allow best choice of n_components
    # TODO: add error handling
    return transformed

#### Fetch top N similar products

In [12]:
def similar_products(corpus, query, vectors, top_n=0):
    cosine_similarities = linear_kernel(query, vectors).flatten().argsort()
    return corpus[cosine_similarities[-top_n:]]

#### Run

In [13]:
def run(corpus, query=None, top_n=3):
    vectors = vectorize(corpus, refit=True if query is None else False)
    if query:
        query = vectorize([query])
        return similar_products(corpus, query, vectors, top_n=top_n)
    else:
        return "Model Updated."

#### Test

In [15]:
query = 'FULL ENGLISH BREAKFAST PLATE'

run(corpus)

C:\Users\felipe.c.rocha\Anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


'Model Updated.'

In [ ]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    # evaluate model with ranking metrics
    return


if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(42)

    # Read file from the URL
    csv_url = (
        "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/data/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha = float(sys.argv[1]) if len(sys.argv) > 1 else 0.5
    l1_ratio = float(sys.argv[2]) if len(sys.argv) > 2 else 0.5

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha={:f}, l1_ratio={:f}):".format(alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
        else:
            mlflow.sklearn.log_model(lr, "model")